# Oclusão

In [1]:
import numpy as np
import pickle
import json

In [2]:
from dataPreprocess import prepare_data, make_and_save_dict, map_words_and_tags, pad_seq, get_sets

Using TensorFlow backend.


In [3]:
tag2idx = {'I-PER': 0,
           'I-MISC': 1,
           'I-LOC': 2,
           'B-LOC': 3,
           'I-ORG': 4,
           'O': 5,
           'B-ORG': 6,
           'B-MISC': 7,
           'B-PER': 8,
          }
num_tags = len(tag2idx)
tags_set = get_sets(tag2idx)

In [4]:
# import word dict and tag dict
word_dict2 = '../model_hu/word_dict2.pkl'
with open(word_dict2, 'rb') as f:
    word2idx = pickle.load(f)
num_words = len(word2idx)
words_set = get_sets(word2idx)
words_set = {}
for wd in word2idx:
    words_set.update({word2idx[wd]: wd})
len(word2idx)

26870

## Import Dataset

In [5]:
# dataset
max_len = 75
num_tags = len(tags_set)
teste = '../conll2003/test.txt'
X_te, y_te = prepare_data(teste, num_tags, max_len, word2idx, tag2idx)
treino = '../conll2003/train.txt'
X_tr, y_tr = prepare_data(treino, num_tags, max_len, word2idx, tag2idx)
len(X_tr), len(X_te)

C:\Users\gfra2\anaconda3\envs\projeto_20220828\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(14041, 3453)

## Import Models

In [6]:
from keras.models import load_model

In [7]:
model1 = '../project_models/828model_glove.h5'
model2 = '../project_models/828model_glove.h10'
model3 = '../project_models/828model_glove.h20'
model4 = '../project_models/828model_glove.h50'
model5 = '../project_models/828model_glove.h100'
models_folders = [model1, model2, model3, model4, model5]

In [8]:
models = []
for m in models_folders:
    models.append(load_model(m))

# Oclusion Tests

In [9]:
from ocl_cal import oclusion

## Test Sent 1

## Test Sent B-PER 1

In [10]:
exp = 307
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

tour manager clive lloyd on wednesday apologised for lara 's behaviour in confronting australia coach geoff marsh in the opposition dressing room to protest against his dismissal in the second test on tuesday .


In [21]:
word_idx_target = 2
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: clive          
Tag : B-PER


In [22]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [23]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- tour           | O       -  O       -  O       -  O       -  O       -  O     
 1- manager        | O       -  O       -  O       -  O       -  O       -  O     
 2- clive          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 3- lloyd          | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 4- on             | O       -  O       -  O       -  O       -  O       -  O     
 5- wednesday      | O       -  O       -  O       -  O       -  O       -  O     
 6- apologised     | O       -  O       -  O       -  O       -  O       -  O     
 7- for            | O       -  O       -  O       -  O       -  O       -  O     
 8- lara           | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 9- 's             | O       -  O       -  O       -  O       -  O       -  O     
10- behaviour      | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [24]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [25]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: clive          
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- clive      (9.48E-01)|1- clive      (9.82E-01)|1- clive      (9.82E-01)|1- clive      (1.00E+00)|1- clive      (1.00E+00)
2- lloyd      (2.87E-01)|2- lloyd      (4.37E-01)|2- lloyd      (7.81E-01)|2- lloyd      (8.91E-01)|2- lloyd      (9.66E-01)
3- manager    (1.77E-02)|3- manager    (3.41E-03)|3- lara       (1.61E-04)|3- lara       (4.45E-04)|3- lara       (1.31E-05)
4- apologised (9.28E-04)|4- apologised (1.72E-03)|4- manager    (1.14E-04)|4- for        (4.02E-06)|4- for        (6.13E-07)
5- lara       (4.93E-04)|5- tour       (4.15E-04)|5- apologised (8.61E-05)|5- wednesday  (3.67E-06)|5- geoff      (2.38E-07)


## Test Sent B-PER 2

In [10]:
exp = 307
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

tour manager clive lloyd on wednesday apologised for lara 's behaviour in confronting australia coach geoff marsh in the opposition dressing room to protest against his dismissal in the second test on tuesday .


In [26]:
word_idx_target = 8
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: lara           
Tag : B-PER


In [27]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [28]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- tour           | O       -  O       -  O       -  O       -  O       -  O     
 1- manager        | O       -  O       -  O       -  O       -  O       -  O     
 2- clive          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 3- lloyd          | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 4- on             | O       -  O       -  O       -  O       -  O       -  O     
 5- wednesday      | O       -  O       -  O       -  O       -  O       -  O     
 6- apologised     | O       -  O       -  O       -  O       -  O       -  O     
 7- for            | O       -  O       -  O       -  O       -  O       -  O     
 8- lara           | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 9- 's             | O       -  O       -  O       -  O       -  O       -  O     
10- behaviour      | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [29]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [30]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: lara           
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- lara       (9.65E-01)|1- lara       (9.86E-01)|1- lara       (9.87E-01)|1- lara       (9.98E-01)|1- lara       (1.00E+00)
2- apologised (4.49E-02)|2- apologised (4.28E-02)|2- clive      (7.07E-03)|2- apologised (8.05E-05)|2- apologised (7.08E-06)
3- clive      (3.24E-03)|3- clive      (2.34E-03)|3- apologised (4.95E-03)|3- clive      (2.10E-05)|3- for        (3.56E-07)
4- geoff      (7.74E-04)|4- geoff      (7.35E-04)|4- for        (2.41E-04)|4- wednesday  (1.88E-05)|4- wednesday  (3.56E-07)
5- for        (7.62E-04)|5- for        (6.79E-04)|5- wednesday  (1.90E-04)|5- for        (5.13E-06)|5- 's         (1.68E-07)


## Test Sent B-PER 3

In [31]:
exp = 310
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

australia went on to win the match at the sydney cricket ground by 124 runs to take a two-nil lead in the five-test series after lara failed in both innings .


In [35]:
word_idx_target = 25
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: lara           
Tag : B-PER


In [36]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [37]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- australia      | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- went           | O       -  O       -  O       -  O       -  O       -  O     
 2- on             | O       -  O       -  O       -  O       -  O       -  O     
 3- to             | O       -  O       -  O       -  O       -  O       -  O     
 4- win            | O       -  O       -  O       -  O       -  O       -  O     
 5- the            | O       -  O       -  O       -  O       -  O       -  O     
 6- match          | O       -  O       -  O       -  O       -  O       -  O     
 7- at             | O       -  O       -  O       -  O       -  O       -  O     
 8- the            | O       -  O       -  O       -  O       -  O       -  O     
 9- sydney         | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
10- cricket        | I-LOC   -  I-LOC   -  O       -  O       -  O       -  O     
11- 

In [38]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [39]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: lara           
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- lara       (9.43E-01)|1- lara       (9.72E-01)|1- lara       (9.89E-01)|1- lara       (9.97E-01)|1- lara       (1.00E+00)
2- two-nil    (1.14E-02)|2- five-test  (4.01E-03)|2- five-test  (5.19E-04)|2- five-test  (4.02E-05)|2- five-test  (2.32E-06)
3- five-test  (3.82E-03)|3- two-nil    (2.53E-03)|3- two-nil    (1.48E-04)|3- two-nil    (1.73E-06)|3- a          (1.09E-07)
4- after      (1.52E-03)|4- after      (2.74E-04)|4- a          (1.08E-04)|4- series     (1.33E-06)|4- 124        (5.26E-08)
5- both       (4.40E-04)|5- both       (1.90E-04)|5- innings    (5.69E-05)|5- a          (9.06E-07)|5- innings    (2.83E-08)


## Test Sent B-LOC 1

In [136]:
exp = 310
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

australia went on to win the match at the sydney cricket ground by 124 runs to take a two-nil lead in the five-test series after lara failed in both innings .


In [40]:
word_idx_target = 9
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: sydney         
Tag : B-LOC


In [41]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [42]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- australia      | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- went           | O       -  O       -  O       -  O       -  O       -  O     
 2- on             | O       -  O       -  O       -  O       -  O       -  O     
 3- to             | O       -  O       -  O       -  O       -  O       -  O     
 4- win            | O       -  O       -  O       -  O       -  O       -  O     
 5- the            | O       -  O       -  O       -  O       -  O       -  O     
 6- match          | O       -  O       -  O       -  O       -  O       -  O     
 7- at             | O       -  O       -  O       -  O       -  O       -  O     
 8- the            | O       -  O       -  O       -  O       -  O       -  O     
 9- sydney         | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
10- cricket        | I-LOC   -  I-LOC   -  O       -  O       -  O       -  O     
11- 

In [43]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [44]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: sydney         
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- sydney     (8.36E-01)|1- sydney     (9.20E-01)|1- sydney     (9.71E-01)|1- sydney     (9.95E-01)|1- sydney     (1.00E+00)
2- cricket    (2.67E-01)|2- cricket    (4.08E-02)|2- the        (1.22E-01)|2- the        (2.63E-01)|2- the        (3.14E-02)
3- the        (3.44E-02)|3- the        (9.10E-03)|3- cricket    (1.58E-02)|3- cricket    (4.77E-04)|3- cricket    (5.64E-05)
4- australia  (1.12E-03)|4- australia  (1.20E-04)|4- australia  (2.74E-05)|4- australia  (1.51E-04)|4- five-test  (7.23E-07)
5- at         (2.38E-04)|5- at         (4.00E-05)|5- at         (1.08E-05)|5- five-test  (7.59E-06)|5- australia  (1.19E-07)


## Test Sent B-LOC 2

In [139]:
exp = 310
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

australia went on to win the match at the sydney cricket ground by 124 runs to take a two-nil lead in the five-test series after lara failed in both innings .


In [50]:
word_idx_target = 0
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: australia      
Tag : B-LOC


In [51]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [52]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- australia      | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- went           | O       -  O       -  O       -  O       -  O       -  O     
 2- on             | O       -  O       -  O       -  O       -  O       -  O     
 3- to             | O       -  O       -  O       -  O       -  O       -  O     
 4- win            | O       -  O       -  O       -  O       -  O       -  O     
 5- the            | O       -  O       -  O       -  O       -  O       -  O     
 6- match          | O       -  O       -  O       -  O       -  O       -  O     
 7- at             | O       -  O       -  O       -  O       -  O       -  O     
 8- the            | O       -  O       -  O       -  O       -  O       -  O     
 9- sydney         | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
10- cricket        | I-LOC   -  I-LOC   -  O       -  O       -  O       -  O     
11- 

In [53]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [54]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: australia      
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- australia  (9.37E-01)|1- australia  (9.85E-01)|1- australia  (9.95E-01)|1- australia  (9.99E-01)|1- australia  (1.00E+00)
2- went       (2.06E-03)|2- the        (1.55E-03)|2- the        (1.45E-02)|2- the        (3.82E-02)|2- the        (9.79E-03)
3- sydney     (7.65E-04)|3- sydney     (1.04E-03)|3- sydney     (2.45E-03)|3- sydney     (1.51E-03)|3- sydney     (3.16E-05)
4- cricket    (2.85E-04)|4- went       (1.04E-04)|4- went       (5.69E-06)|4- cricket    (1.76E-06)|4- five-test  (2.10E-07)
5- the        (2.55E-04)|5- on         (4.76E-06)|5- five-test  (3.26E-06)|5- five-test  (1.22E-06)|5- went       (1.69E-07)


## Test Sent B-PER 4

In [64]:
exp = 300
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

west indies batsman brian lara suffered another blow to his australian tour , after already being disciplined for misconduct , when he was dismissed cheaply in the first limited overs match against australia on friday .


In [65]:
word_idx_target = 3
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: brian          
Tag : B-PER


In [66]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [67]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- west           | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- indies         | I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC 
 2- batsman        | O       -  O       -  O       -  O       -  O       -  O     
 3- brian          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 4- lara           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 5- suffered       | O       -  O       -  O       -  O       -  O       -  O     
 6- another        | O       -  O       -  O       -  O       -  O       -  O     
 7- blow           | O       -  O       -  O       -  O       -  O       -  O     
 8- to             | O       -  O       -  O       -  O       -  O       -  O     
 9- his            | O       -  O       -  O       -  O       -  O       -  O     
10- australian     | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
11- 

In [68]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [69]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: brian          
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- brian      (9.94E-01)|1- brian      (1.00E+00)|1- brian      (1.00E+00)|1- brian      (1.00E+00)|1- brian      (1.00E+00)
2- batsman    (7.16E-01)|2- lara       (8.36E-01)|2- lara       (9.44E-01)|2- lara       (9.97E-01)|2- lara       (9.91E-01)
3- lara       (2.88E-01)|3- batsman    (1.88E-01)|3- batsman    (4.21E-02)|3- batsman    (6.29E-04)|3- batsman    (1.72E-04)
4- indies     (1.76E-03)|4- indies     (1.14E-04)|4- west       (4.98E-04)|4- west       (3.19E-04)|4- west       (2.65E-05)
5- west       (2.88E-04)|5- west       (5.11E-05)|5- misconduct (3.20E-06)|5- blow       (1.05E-06)|5- australia  (1.38E-08)


## Test Sent B-LOC 3

In [70]:
exp = 300
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

west indies batsman brian lara suffered another blow to his australian tour , after already being disciplined for misconduct , when he was dismissed cheaply in the first limited overs match against australia on friday .


In [74]:
word_idx_target = 0
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: west           
Tag : B-LOC


In [75]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [76]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- west           | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- indies         | I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC 
 2- batsman        | O       -  O       -  O       -  O       -  O       -  O     
 3- brian          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 4- lara           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 5- suffered       | O       -  O       -  O       -  O       -  O       -  O     
 6- another        | O       -  O       -  O       -  O       -  O       -  O     
 7- blow           | O       -  O       -  O       -  O       -  O       -  O     
 8- to             | O       -  O       -  O       -  O       -  O       -  O     
 9- his            | O       -  O       -  O       -  O       -  O       -  O     
10- australian     | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
11- 

In [77]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [78]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: west           
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- west       (8.57E-01)|1- west       (9.24E-01)|1- indies     (8.62E-01)|1- west       (9.49E-01)|1- west       (9.84E-01)
2- indies     (4.58E-01)|2- indies     (7.30E-01)|2- west       (8.18E-01)|2- indies     (9.12E-01)|2- indies     (6.49E-01)
3- tour       (2.70E-05)|3- australian (9.75E-05)|3- australian (3.90E-05)|3- australia  (4.79E-05)|3- australia  (1.03E-04)
4- blow       (2.03E-05)|4- australia  (3.46E-06)|4- australia  (3.87E-06)|4- australian (1.67E-05)|4- australian (3.83E-06)
5- australian (1.66E-05)|5- blow       (1.35E-06)|5- against    (8.55E-07)|5- against    (1.07E-07)|5- against    (1.02E-08)


## Test Sent B-LOC 4

In [79]:
exp = 300
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

west indies batsman brian lara suffered another blow to his australian tour , after already being disciplined for misconduct , when he was dismissed cheaply in the first limited overs match against australia on friday .


In [81]:
word_idx_target = 32
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: australia      
Tag : B-LOC


In [82]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [83]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- west           | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- indies         | I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC 
 2- batsman        | O       -  O       -  O       -  O       -  O       -  O     
 3- brian          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 4- lara           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 5- suffered       | O       -  O       -  O       -  O       -  O       -  O     
 6- another        | O       -  O       -  O       -  O       -  O       -  O     
 7- blow           | O       -  O       -  O       -  O       -  O       -  O     
 8- to             | O       -  O       -  O       -  O       -  O       -  O     
 9- his            | O       -  O       -  O       -  O       -  O       -  O     
10- australian     | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
11- 

In [84]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [85]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: australia      
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- australia  (8.97E-01)|1- australia  (9.83E-01)|1- australia  (9.93E-01)|1- australia  (9.97E-01)|1- australia  (9.99E-01)
2- against    (2.16E-02)|2- against    (3.34E-03)|2- against    (1.91E-03)|2- west       (9.33E-05)|2- west       (3.42E-05)
3- friday     (1.03E-03)|3- west       (3.01E-04)|3- australian (3.93E-05)|3- against    (8.58E-06)|3- against    (4.40E-07)
4- on         (6.38E-04)|4- friday     (1.45E-04)|4- west       (3.52E-05)|4- australian (4.84E-06)|4- australian (6.04E-08)
5- first      (8.39E-05)|5- indies     (1.01E-04)|5- friday     (4.08E-06)|5- friday     (2.45E-07)|5- friday     (2.60E-08)


## Test Sent B-LOC 5

In [130]:
exp = 5
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

china controlled most of the match and saw several chances missed until the 78th minute when uzbek striker igor shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing chinese keeper and into an empty net .


In [131]:
word_idx_target = 0
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: china          
Tag : B-LOC


In [132]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [133]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- china          | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- controlled     | O       -  O       -  O       -  O       -  O       -  O     
 2- most           | O       -  O       -  O       -  O       -  O       -  O     
 3- of             | O       -  O       -  O       -  O       -  O       -  O     
 4- the            | O       -  O       -  O       -  O       -  O       -  O     
 5- match          | O       -  O       -  O       -  O       -  O       -  O     
 6- and            | O       -  O       -  O       -  O       -  O       -  O     
 7- saw            | O       -  O       -  O       -  O       -  O       -  O     
 8- several        | O       -  O       -  O       -  O       -  O       -  O     
 9- chances        | O       -  O       -  O       -  O       -  O       -  O     
10- missed         | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [134]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [135]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: china          
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- china      (9.54E-01)|1- china      (9.90E-01)|1- china      (9.98E-01)|1- china      (1.00E+00)|1- china      (1.00E+00)
2- controlled (2.37E-03)|2- controlled (7.28E-04)|2- controlled (1.75E-04)|2- uzbek      (5.33E-05)|2- uzbek      (2.48E-05)
3- most       (1.01E-04)|3- most       (2.62E-05)|3- uzbek      (2.11E-05)|3- controlled (4.19E-05)|3- controlled (1.46E-06)
4- the        (5.23E-05)|4- uzbek      (8.19E-06)|4- most       (5.83E-06)|4- chinese    (5.10E-06)|4- chinese    (5.41E-07)
5- of         (4.77E-05)|5- chinese    (5.42E-06)|5- chinese    (6.55E-07)|5- most       (1.36E-07)|5- most       (5.66E-09)


## Test Sent B-PER 5

In [123]:
exp = 5
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

china controlled most of the match and saw several chances missed until the 78th minute when uzbek striker igor shkvyrin took advantage of a misdirected defensive header to lob the ball over the advancing chinese keeper and into an empty net .


In [127]:
word_idx_target = 18
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: igor           
Tag : B-PER


In [125]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [126]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- china          | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- controlled     | O       -  O       -  O       -  O       -  O       -  O     
 2- most           | O       -  O       -  O       -  O       -  O       -  O     
 3- of             | O       -  O       -  O       -  O       -  O       -  O     
 4- the            | O       -  O       -  O       -  O       -  O       -  O     
 5- match          | O       -  O       -  O       -  O       -  O       -  O     
 6- and            | O       -  O       -  O       -  O       -  O       -  O     
 7- saw            | O       -  O       -  O       -  O       -  O       -  O     
 8- several        | O       -  O       -  O       -  O       -  O       -  O     
 9- chances        | O       -  O       -  O       -  O       -  O       -  O     
10- missed         | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [128]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [129]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: igor           
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- igor       (9.44E-01)|1- igor       (9.99E-01)|1- igor       (1.00E+00)|1- igor       (1.00E+00)|1- igor       (1.00E+00)
2- striker    (1.04E-01)|2- shkvyrin   (5.60E-01)|2- shkvyrin   (9.44E-01)|2- shkvyrin   (9.98E-01)|2- shkvyrin   (1.00E+00)
3- shkvyrin   (5.75E-02)|3- striker    (1.90E-02)|3- striker    (3.68E-04)|3- striker    (8.85E-05)|3- header     (1.05E-05)
4- keeper     (1.68E-04)|4- defensive  (7.53E-05)|4- defensive  (1.77E-05)|4- keeper     (4.05E-06)|4- striker    (1.77E-06)
5- lob        (1.16E-04)|5- lob        (1.91E-05)|5- uzbek      (9.56E-06)|5- header     (1.59E-06)|5- uzbek      (6.58E-07)


## Test Sent I-PER 1

In [10]:
exp = 307
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

tour manager clive lloyd on wednesday apologised for lara 's behaviour in confronting australia coach geoff marsh in the opposition dressing room to protest against his dismissal in the second test on tuesday .


In [11]:
word_idx_target = 3
tag_target = 'I-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: lloyd          
Tag : I-PER


In [12]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [13]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- tour           | O       -  O       -  O       -  O       -  O       -  O     
 1- manager        | O       -  O       -  O       -  O       -  O       -  O     
 2- clive          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 3- lloyd          | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 4- on             | O       -  O       -  O       -  O       -  O       -  O     
 5- wednesday      | O       -  O       -  O       -  O       -  O       -  O     
 6- apologised     | O       -  O       -  O       -  O       -  O       -  O     
 7- for            | O       -  O       -  O       -  O       -  O       -  O     
 8- lara           | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 9- 's             | O       -  O       -  O       -  O       -  O       -  O     
10- behaviour      | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [14]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [15]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: lloyd          
Tag : I-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- lloyd      (9.70E-01)|1- lloyd      (9.81E-01)|1- lloyd      (9.57E-01)|1- lloyd      (1.00E+00)|1- lloyd      (1.00E+00)
2- clive      (3.94E-01)|2- clive      (1.93E-01)|2- clive      (6.21E-03)|2- on         (1.12E-03)|2- clive      (6.73E-05)
3- on         (1.70E-03)|3- on         (9.60E-04)|3- on         (1.51E-03)|3- clive      (1.83E-04)|3- on         (5.98E-05)
4- manager    (5.96E-04)|4- manager    (1.88E-04)|4- room       (9.22E-05)|4- 's         (5.79E-06)|4- 's         (5.83E-06)
5- apologised (1.88E-04)|5- 's         (1.27E-04)|5- manager    (3.24E-05)|5- lara       (3.46E-07)|5- lara       (1.55E-07)


## Test Sent I-PER 2

In [55]:
exp = 300
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

west indies batsman brian lara suffered another blow to his australian tour , after already being disciplined for misconduct , when he was dismissed cheaply in the first limited overs match against australia on friday .


In [59]:
word_idx_target = 4
tag_target = 'I-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: lara           
Tag : I-PER


In [60]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [61]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- west           | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- indies         | I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC 
 2- batsman        | O       -  O       -  O       -  O       -  O       -  O     
 3- brian          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 4- lara           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 5- suffered       | O       -  O       -  O       -  O       -  O       -  O     
 6- another        | O       -  O       -  O       -  O       -  O       -  O     
 7- blow           | O       -  O       -  O       -  O       -  O       -  O     
 8- to             | O       -  O       -  O       -  O       -  O       -  O     
 9- his            | O       -  O       -  O       -  O       -  O       -  O     
10- australian     | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
11- 

In [62]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [63]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: lara           
Tag : I-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- lara       (9.83E-01)|1- lara       (9.97E-01)|1- lara       (1.00E+00)|1- lara       (1.00E+00)|1- lara       (1.00E+00)
2- brian      (1.91E-01)|2- brian      (1.05E-02)|2- suffered   (1.47E-02)|2- brian      (3.87E-04)|2- suffered   (3.40E-04)
3- batsman    (3.14E-02)|3- suffered   (7.56E-03)|3- brian      (5.15E-03)|3- suffered   (4.41E-05)|3- brian      (1.87E-04)
4- suffered   (3.13E-03)|4- batsman    (3.52E-04)|4- tour       (1.56E-04)|4- tour       (1.88E-06)|4- tour       (8.65E-05)
5- indies     (8.67E-04)|5- indies     (3.34E-04)|5- indies     (1.11E-04)|5- west       (3.51E-07)|5- indies     (4.97E-07)


## Test Sent I-LOC 1

In [137]:
exp = 310
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

australia went on to win the match at the sydney cricket ground by 124 runs to take a two-nil lead in the five-test series after lara failed in both innings .


In [138]:
word_idx_target = 10
tag_target = 'I-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: cricket        
Tag : I-LOC


In [46]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [47]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- australia      | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- went           | O       -  O       -  O       -  O       -  O       -  O     
 2- on             | O       -  O       -  O       -  O       -  O       -  O     
 3- to             | O       -  O       -  O       -  O       -  O       -  O     
 4- win            | O       -  O       -  O       -  O       -  O       -  O     
 5- the            | O       -  O       -  O       -  O       -  O       -  O     
 6- match          | O       -  O       -  O       -  O       -  O       -  O     
 7- at             | O       -  O       -  O       -  O       -  O       -  O     
 8- the            | O       -  O       -  O       -  O       -  O       -  O     
 9- sydney         | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
10- cricket        | I-LOC   -  I-LOC   -  O       -  O       -  O       -  O     
11- 

In [48]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [49]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: cricket        
Tag : I-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- cricket    (5.58E-01)|1- cricket    (3.29E-01)|1- cricket    (3.27E-01)|1- cricket    (2.37E-02)|1- ground     (2.84E-03)
2- sydney     (4.96E-02)|2- sydney     (2.12E-02)|2- ground     (7.54E-03)|2- ground     (2.13E-03)|2- cricket    (8.60E-04)
3- ground     (6.60E-03)|3- ground     (4.63E-03)|3- sydney     (4.75E-03)|3- sydney     (9.07E-05)|3- sydney     (1.26E-06)
4- runs       (4.62E-03)|4- the        (4.50E-04)|4- australia  (6.96E-05)|4- australia  (5.08E-06)|4- went       (4.62E-08)
5- australia  (7.14E-04)|5- runs       (3.15E-04)|5- at         (4.74E-05)|5- failed     (2.87E-07)|5- australia  (3.31E-08)


## Test Sent I-LOC 2

In [140]:
exp = 300
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

west indies batsman brian lara suffered another blow to his australian tour , after already being disciplined for misconduct , when he was dismissed cheaply in the first limited overs match against australia on friday .


In [146]:
word_idx_target = 1
tag_target = 'I-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: indies         
Tag : I-LOC


In [147]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [148]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- west           | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- indies         | I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC   -  I-LOC 
 2- batsman        | O       -  O       -  O       -  O       -  O       -  O     
 3- brian          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 4- lara           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 5- suffered       | O       -  O       -  O       -  O       -  O       -  O     
 6- another        | O       -  O       -  O       -  O       -  O       -  O     
 7- blow           | O       -  O       -  O       -  O       -  O       -  O     
 8- to             | O       -  O       -  O       -  O       -  O       -  O     
 9- his            | O       -  O       -  O       -  O       -  O       -  O     
10- australian     | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
11- 

In [149]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [150]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: indies         
Tag : I-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- indies     (6.59E-01)|1- indies     (8.32E-01)|1- indies     (8.41E-01)|1- indies     (9.49E-01)|1- indies     (5.84E-01)
2- west       (1.90E-01)|2- west       (8.51E-02)|2- west       (3.02E-02)|2- west       (7.47E-03)|2- west       (2.95E-02)
3- batsman    (3.62E-04)|3- batsman    (2.19E-04)|3- batsman    (8.22E-05)|3- tour       (9.47E-05)|3- tour       (9.21E-06)
4- tour       (3.42E-04)|4- tour       (2.50E-05)|4- tour       (3.34E-05)|4- lara       (4.47E-07)|4- lara       (1.82E-08)
5- suffered   (1.01E-05)|5- lara       (3.54E-06)|5- australia  (3.31E-06)|5- australia  (7.48E-08)|5- australia  (5.90E-09)


In [ ]:
19 (borg o) 349 (bloc borg)

## Test Sent I-ORG 1

In [513]:
exp = 405
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

( corrects headline from nba to nhl and corrects team name in second result from la clippers to ny islanders .


In [514]:
word_idx_target = 16
tag_target = 'I-ORG'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: clippers       
Tag : I-ORG


In [515]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [516]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- (              | O       -  O       -  O       -  O       -  O       -  O     
 1- corrects       | O       -  O       -  O       -  O       -  O       -  O     
 2- headline       | O       -  O       -  O       -  O       -  O       -  O     
 3- from           | O       -  O       -  O       -  O       -  O       -  O     
 4- nba            | B-ORG   -  O       -  O       -  O       -  O       -  O     
 5- to             | O       -  O       -  O       -  O       -  O       -  O     
 6- nhl            | B-ORG   -  O       -  O       -  O       -  O       -  O     
 7- and            | O       -  O       -  O       -  O       -  O       -  O     
 8- corrects       | O       -  O       -  O       -  O       -  O       -  O     
 9- team           | O       -  O       -  O       -  O       -  O       -  O     
10- name           | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [517]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [518]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: clippers       
Tag : I-ORG

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- clippers   (8.19E-01)|1- clippers   (9.63E-01)|1- clippers   (9.93E-01)|1- clippers   (9.83E-01)|1- clippers   (5.08E-01)
2- to         (8.57E-02)|2- to         (4.87E-02)|2- to         (2.71E-02)|2- islanders  (1.11E-02)|2- islanders  (2.00E-03)
3- la         (5.46E-02)|3- ny         (3.09E-02)|3- islanders  (1.68E-02)|3- to         (8.62E-05)|3- ny         (5.22E-05)
4- ny         (2.08E-02)|4- islanders  (6.42E-03)|4- ny         (1.00E-02)|4- nba        (2.24E-05)|4- team       (5.23E-07)
5- islanders  (1.62E-03)|5- la         (5.74E-03)|5- la         (2.42E-03)|5- ny         (1.93E-05)|5- to         (5.22E-07)


## Test Sent O - B-PER

In [519]:
exp = 13
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

nader jokhadar had given syria the lead with a well-struck header in the seventh minute .


In [520]:
word_idx_target = 9
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: well-struck    
Tag : O    


In [521]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [522]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- nader          | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 1- jokhadar       | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 2- had            | O       -  O       -  O       -  O       -  O       -  O     
 3- given          | O       -  O       -  O       -  O       -  O       -  O     
 4- syria          | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 5- the            | O       -  O       -  O       -  O       -  O       -  O     
 6- lead           | O       -  O       -  O       -  O       -  O       -  O     
 7- with           | O       -  O       -  O       -  O       -  O       -  O     
 8- a              | O       -  O       -  O       -  O       -  O       -  O     
 9- well-struck    | O       -  B-PER   -  O       -  O       -  O       -  O     
10- header         | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [523]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [524]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: well-struck    
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- well-struck(4.07E-01)|1- well-struck(7.02E-01)|1- well-struck(9.12E-01)|1- well-struck(9.52E-01)|1- well-struck(9.98E-01)
2- header     (2.21E-02)|2- a          (1.18E-03)|2- a          (4.14E-03)|2- a          (2.21E-04)|2- a          (1.33E-04)
3- jokhadar   (2.94E-03)|3- jokhadar   (9.79E-04)|3- nader      (9.60E-04)|3- nader      (6.04E-05)|3- nader      (2.86E-06)
4- syria      (8.25E-04)|4- syria      (5.68E-04)|4- jokhadar   (4.49E-04)|4- jokhadar   (5.46E-05)|4- jokhadar   (1.76E-06)
5- nader      (2.87E-04)|5- header     (4.78E-04)|5- syria      (1.91E-04)|5- syria      (1.86E-05)|5- minute     (1.19E-07)


In [525]:
tag_target = 'B-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: well-struck    
Tag : B-PER


In [526]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [527]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: well-struck    
Tag : B-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- well-struck(5.51E-01)|1- well-struck(2.87E-01)|1- well-struck(7.35E-02)|1- well-struck(9.12E-03)|1- well-struck(7.40E-04)
2- jokhadar   (2.89E-03)|2- a          (1.08E-03)|2- a          (4.02E-03)|2- a          (2.20E-04)|2- a          (1.32E-04)
3- header     (1.92E-03)|3- jokhadar   (5.01E-04)|3- nader      (1.71E-03)|3- nader      (7.81E-05)|3- nader      (3.70E-06)
4- nader      (6.41E-04)|4- nader      (4.01E-04)|4- syria      (8.52E-05)|4- syria      (1.06E-05)|4- jokhadar   (5.15E-07)
5- a          (1.18E-04)|5- syria      (8.01E-05)|5- jokhadar   (4.58E-05)|5- jokhadar   (4.18E-06)|5- syria      (2.37E-07)


In [528]:
word_idx_target = 10
tag_target = 'I-PER'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: header         
Tag : I-PER


In [529]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [530]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: header         
Tag : I-PER

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- header     (2.43E-02)|1- jokhadar   (8.34E-04)|1- well-struck(2.88E-04)|1- jokhadar   (3.57E-04)|1- jokhadar   (8.73E-04)
2- jokhadar   (1.54E-03)|2- header     (4.07E-04)|2- jokhadar   (7.04E-05)|2- well-struck(1.37E-04)|2- well-struck(1.36E-05)
3- well-struck(1.96E-04)|3- well-struck(1.61E-04)|3- header     (3.28E-05)|3- nader      (5.76E-06)|3- nader      (2.41E-06)
4- minute     (3.97E-05)|4- nader      (2.44E-05)|4- in         (2.05E-06)|4- header     (1.20E-07)|4- the        (2.31E-10)
5- nader      (3.31E-05)|5- in         (4.83E-06)|5- nader      (1.85E-06)|5- syria      (3.25E-08)|5- syria      (2.16E-10)


## Test Sent O - B-LOC

In [531]:
exp = 16
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

japan coach shu kamo said : ' ' the syrian own goal proved lucky for us .


In [532]:
word_idx_target = 15
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: us             
Tag : O    


In [533]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [534]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- japan          | B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC   -  B-LOC 
 1- coach          | O       -  O       -  O       -  O       -  O       -  O     
 2- shu            | B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER   -  B-PER 
 3- kamo           | I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER   -  I-PER 
 4- said           | O       -  O       -  O       -  O       -  O       -  O     
 5- :              | O       -  O       -  O       -  O       -  O       -  O     
 6- '              | O       -  O       -  O       -  O       -  O       -  O     
 7- '              | O       -  O       -  O       -  O       -  O       -  O     
 8- the            | O       -  O       -  O       -  O       -  O       -  O     
 9- syrian         | B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
10- own            | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [535]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [536]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: us             
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- us         (3.80E-01)|1- us         (7.75E-01)|1- us         (9.60E-01)|1- us         (9.73E-01)|1- us         (9.98E-01)
2- .          (6.31E-03)|2- .          (1.35E-03)|2- :          (7.40E-04)|2- :          (7.24E-03)|2- said       (2.15E-02)
3- lucky      (4.39E-03)|3- own        (9.19E-04)|3- .          (5.74E-04)|3- said       (1.13E-03)|3- :          (2.84E-04)
4- '          (1.44E-03)|4- lucky      (7.54E-04)|4- coach      (2.03E-04)|4- kamo       (2.42E-05)|4- own        (1.03E-05)
5- '          (7.91E-04)|5- :          (5.61E-04)|5- goal       (2.00E-04)|5- lucky      (2.21E-05)|5- goal       (3.81E-06)


In [537]:
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: us             
Tag : B-LOC


In [538]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [539]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: us             
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- us         (4.90E-01)|1- us         (1.79E-01)|1- us         (3.27E-02)|1- us         (2.57E-02)|1- us         (2.08E-03)
2- lucky      (2.03E-04)|2- :          (4.01E-04)|2- :          (4.83E-04)|2- :          (7.12E-03)|2- :          (2.18E-04)
3- syrian     (1.92E-04)|3- syrian     (1.14E-04)|3- syrian     (7.26E-05)|3- syrian     (3.65E-04)|3- syrian     (5.60E-06)
4- .          (7.70E-05)|4- japan      (8.40E-05)|4- japan      (2.09E-05)|4- japan      (3.88E-05)|4- japan      (3.58E-07)
5- japan      (4.05E-05)|5- shu        (2.90E-05)|5- .          (7.79E-06)|5- .          (3.20E-06)|5- .          (2.36E-07)


## Test Sent B-ORG - O

In [540]:
exp = 414
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

nfl american football-colts clobber eagles to stay in playoff hunt .


In [541]:
word_idx_target = 0
tag_target = 'B-ORG'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: nfl            
Tag : B-ORG


In [542]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [543]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- nfl            | B-ORG   -  O       -  B-ORG   -  O       -  O       -  O     
 1- american       | O       -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC  -  B-MISC
 2- football-colts | O       -  O       -  O       -  O       -  O       -  O     
 3- clobber        | O       -  O       -  O       -  O       -  O       -  O     
 4- eagles         | B-ORG   -  B-ORG   -  B-ORG   -  B-ORG   -  B-ORG   -  O     
 5- to             | O       -  O       -  O       -  O       -  O       -  O     
 6- stay           | O       -  O       -  O       -  O       -  O       -  O     
 7- in             | O       -  O       -  O       -  O       -  O       -  O     
 8- playoff        | O       -  O       -  O       -  O       -  O       -  O     
 9- hunt           | O       -  O       -  O       -  O       -  O       -  O     
10- .              | O       -  O       -  O       -  O       -  O       -  O     


In [544]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [545]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: nfl            
Tag : B-ORG

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- nfl        (2.47E-01)|1- nfl        (4.85E-01)|1- nfl        (1.87E-01)|1- eagles     (9.87E-02)|1- eagles     (1.87E-02)
2- clobber    (1.26E-01)|2- eagles     (9.49E-02)|2- eagles     (7.44E-03)|2- nfl        (3.77E-02)|2- nfl        (1.13E-03)
3- eagles     (1.22E-01)|3- american   (2.38E-02)|3- clobber    (3.26E-03)|3- clobber    (1.58E-04)|3- clobber    (4.48E-05)
4- football-colts(2.13E-02)|4- football-colts(7.63E-03)|4- american   (2.29E-03)|4- american   (6.77E-05)|4- football-colts(8.77E-07)
5- american   (1.22E-02)|5- clobber    (6.64E-03)|5- football-colts(1.84E-03)|5- football-colts(3.27E-06)|5- american   (3.87E-07)


In [546]:
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: nfl            
Tag : O    


In [547]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [548]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: nfl            
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- nfl        (5.55E-01)|1- nfl        (3.23E-01)|1- nfl        (4.77E-01)|1- nfl        (8.70E-01)|1- nfl        (9.50E-01)
2- clobber    (1.26E-01)|2- eagles     (9.97E-02)|2- eagles     (1.18E-02)|2- eagles     (1.04E-01)|2- clobber    (1.10E-02)
3- eagles     (1.13E-01)|3- american   (2.36E-02)|3- football-colts(7.47E-03)|3- clobber    (1.14E-03)|3- eagles     (7.08E-03)
4- football-colts(2.24E-02)|4- football-colts(2.07E-02)|4- clobber    (7.21E-03)|4- football-colts(1.89E-04)|4- football-colts(1.50E-04)
5- hunt       (3.49E-03)|5- clobber    (1.04E-02)|5- american   (8.32E-04)|5- playoff    (1.17E-04)|5- american   (3.93E-05)


In [549]:
word_idx_target = 3
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: clobber        
Tag : O    


In [550]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [551]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: clobber        
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- football-colts(7.21E-01)|1- clobber    (9.62E-01)|1- clobber    (9.78E-01)|1- clobber    (1.00E+00)|1- clobber    (9.99E-01)
2- clobber    (6.64E-01)|2- football-colts(4.69E-01)|2- football-colts(2.33E-01)|2- eagles     (2.07E-01)|2- eagles     (2.72E-01)
3- eagles     (2.25E-01)|3- eagles     (2.33E-01)|3- nfl        (8.37E-02)|3- nfl        (5.92E-02)|3- nfl        (2.96E-02)
4- nfl        (5.56E-02)|4- american   (6.95E-03)|4- eagles     (2.79E-03)|4- football-colts(4.46E-03)|4- football-colts(1.11E-03)
5- hunt       (4.17E-03)|5- nfl        (2.07E-03)|5- hunt       (7.37E-04)|5- playoff    (2.50E-04)|5- american   (3.94E-05)


## Test Sent B-ORG - O

In [552]:
exp = 405
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

( corrects headline from nba to nhl and corrects team name in second result from la clippers to ny islanders .


In [553]:
word_idx_target = 4
tag_target = 'B-ORG'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: nba            
Tag : B-ORG


In [554]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [555]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- (              | O       -  O       -  O       -  O       -  O       -  O     
 1- corrects       | O       -  O       -  O       -  O       -  O       -  O     
 2- headline       | O       -  O       -  O       -  O       -  O       -  O     
 3- from           | O       -  O       -  O       -  O       -  O       -  O     
 4- nba            | B-ORG   -  O       -  O       -  O       -  O       -  O     
 5- to             | O       -  O       -  O       -  O       -  O       -  O     
 6- nhl            | B-ORG   -  O       -  O       -  O       -  O       -  O     
 7- and            | O       -  O       -  O       -  O       -  O       -  O     
 8- corrects       | O       -  O       -  O       -  O       -  O       -  O     
 9- team           | O       -  O       -  O       -  O       -  O       -  O     
10- name           | O       -  O       -  O       -  O       -  O       -  O     
11- 

In [556]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [557]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: nba            
Tag : B-ORG

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- nba        (3.68E-02)|1- nba        (1.07E-01)|1- nba        (7.29E-02)|1- nhl        (1.60E-02)|1- nhl        (1.21E-02)
2- corrects   (7.45E-03)|2- nhl        (1.42E-02)|2- nhl        (7.25E-03)|2- ny         (1.10E-02)|2- ny         (6.48E-03)
3- nhl        (5.96E-03)|3- headline   (4.93E-04)|3- la         (9.05E-04)|3- nba        (3.45E-03)|3- la         (4.81E-04)
4- ny         (1.13E-03)|4- clippers   (4.18E-04)|4- ny         (4.37E-04)|4- la         (3.38E-03)|4- nba        (6.92E-05)
5- headline   (9.05E-04)|5- corrects   (2.27E-04)|5- corrects   (3.70E-04)|5- islanders  (3.21E-05)|5- islanders  (3.70E-05)


In [558]:
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: nba            
Tag : O    


In [559]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [560]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: nba            
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- nba        (9.35E-01)|1- nba        (8.67E-01)|1- nba        (9.04E-01)|1- nba        (9.95E-01)|1- nba        (1.00E+00)
2- nhl        (1.34E-02)|2- nhl        (1.29E-02)|2- to         (1.94E-03)|2- nhl        (1.47E-02)|2- nhl        (9.22E-03)
3- corrects   (1.24E-02)|3- to         (2.26E-03)|3- nhl        (9.99E-04)|3- ny         (1.11E-02)|3- ny         (6.03E-03)
4- corrects   (1.01E-02)|4- team       (1.08E-03)|4- la         (8.10E-04)|4- la         (7.18E-04)|4- la         (3.63E-04)
5- to         (5.32E-03)|5- to         (9.10E-04)|5- corrects   (6.39E-04)|5- name       (2.00E-04)|5- islanders  (1.08E-04)


## Test Sent B-LOC - O

In [587]:
exp = 555
test_sent = X_te[exp]
sentence_str = []
for w in test_sent:
    if w == 0:
        break
    sentence_str.append(words_set[w])
print(' '.join(sentence_str))

end_sent = np.where(test_sent == 0)[0][0]
test_true = np.argmax(y_te[exp], -1)

league game on thursday ( home team in caps ) :


In [589]:
pf = []
for model in models:
    p = model.predict(np.array([test_sent]))
    pf.append(np.argmax(p, axis=-1))

In [590]:
seq_len=0
print("{:2}  {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
    "", "Word", "True", "Pred1", "Pred2", "Pred3", "Pred4", "Pred5"))
for w, t, pred1, pred2, pred3, pred4, pred5 in zip(X_te[exp], test_true, pf[0][0], pf[1][0], pf[2][0], pf[3][0], pf[4][0]):
    if w != 0:
        seq_len += 1
        print("{:2}- {:15}| {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}  -  {:6}".format(
            seq_len - 1, words_set[w], tags_set[t],
            tags_set[pred1], tags_set[pred2], tags_set[pred3], tags_set[pred4], tags_set[pred5]))

    Word           | True    -  Pred1   -  Pred2   -  Pred3   -  Pred4   -  Pred5 
 0- league         | B-LOC   -  O       -  O       -  O       -  O       -  O     
 1- game           | O       -  O       -  O       -  O       -  O       -  O     
 2- on             | O       -  O       -  O       -  O       -  O       -  O     
 3- thursday       | O       -  O       -  O       -  O       -  O       -  O     
 4- (              | O       -  O       -  O       -  O       -  O       -  O     
 5- home           | O       -  O       -  O       -  O       -  O       -  O     
 6- team           | O       -  O       -  O       -  O       -  O       -  O     
 7- in             | O       -  O       -  O       -  O       -  O       -  O     
 8- caps           | O       -  O       -  O       -  O       -  O       -  O     
 9- )              | O       -  O       -  O       -  O       -  O       -  O     
10- :              | O       -  O       -  O       -  O       -  O       -  O     


In [595]:
word_idx_target = 0
tag_target = 'B-LOC'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: league         
Tag : B-LOC


In [596]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [597]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: league         
Tag : B-LOC

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- home       (2.18E-03)|1- game       (2.22E-04)|1- caps       (4.95E-04)|1- caps       (9.15E-05)|1- caps       (5.88E-07)
2- caps       (1.78E-03)|2- caps       (1.87E-04)|2- game       (9.32E-05)|2- :          (8.25E-06)|2- game       (5.04E-07)
3- thursday   (1.56E-03)|3- league     (1.09E-04)|3- home       (1.62E-05)|3- game       (2.51E-06)|3- home       (4.99E-07)
4- on         (1.16E-03)|4- thursday   (6.34E-05)|4- thursday   (1.04E-05)|4- home       (4.02E-07)|4- :          (3.83E-07)
5- league     (6.36E-04)|5- on         (3.34E-05)|5- :          (9.38E-06)|5- league     (1.46E-07)|5- league     (1.67E-08)


In [598]:
tag_target = 'O'
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))

Alvo
Word: league         
Tag : O    


In [599]:
sorted_scr_list = []
for model in models:
    sorted_scr_list.append(oclusion (test_sent, tags_set, word_idx_target, tag_target, model))

In [600]:
print_top_rel = 5
i=0
print("Alvo")
print("{:4}: {:15}".format('Word', words_set[X_te[exp][word_idx_target]]))
print("{:4}: {:5}".format('Tag', tag_target))
print()
print("  {:20}  |  {:20}  |  {:20}  |  {:20}  |  {:20}".format("Ranked Relecance 1", "Ranked Relecance 2",
                                                        "Ranked Relecance 3", "Ranked Relecance 4", "Ranked Relecance 5"))
for idx1, idx2, idx3, idx4, idx5 in zip(sorted_scr_list[0].indices, sorted_scr_list[1].indices,
                                        sorted_scr_list[2].indices, sorted_scr_list[3].indices,
                                        sorted_scr_list[4].indices):
    print("{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})|{:1}- {:11}({:.2E})".format(
        i+1, words_set[X_te[exp][idx1]], sorted_scr_list[0].values[i],
        i+1, words_set[X_te[exp][idx2]], sorted_scr_list[1].values[i],
        i+1, words_set[X_te[exp][idx3]], sorted_scr_list[2].values[i],
        i+1, words_set[X_te[exp][idx4]], sorted_scr_list[3].values[i],
        i+1, words_set[X_te[exp][idx5]], sorted_scr_list[4].values[i]))
    if i+1 == print_top_rel:
        break
    i += 1

Alvo
Word: league         
Tag : O    

  Ranked Relecance 1    |  Ranked Relecance 2    |  Ranked Relecance 3    |  Ranked Relecance 4    |  Ranked Relecance 5  
1- league     (8.39E-01)|1- league     (9.72E-01)|1- league     (9.97E-01)|1- league     (1.00E+00)|1- league     (1.00E+00)
2- caps       (3.05E-03)|2- game       (9.41E-03)|2- game       (5.46E-03)|2- caps       (3.80E-04)|2- caps       (6.20E-06)
3- home       (2.89E-03)|3- caps       (4.32E-04)|3- caps       (1.89E-03)|3- :          (9.66E-06)|3- thursday   (2.26E-06)
4- game       (2.88E-03)|4- home       (4.59E-05)|4- thursday   (1.61E-04)|4- thursday   (2.74E-06)|4- home       (1.31E-06)
5- on         (2.04E-03)|5- thursday   (3.31E-05)|5- home       (7.06E-05)|5- )          (1.67E-06)|5- game       (4.77E-07)
